In [20]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import itertools

In [58]:
def addframe(num, grayvalue, img): # for FFT calculation
    # add pixel
    num_insert = int((num-1)/2)
    # create block
    bk_tb = np.zeros((num_insert,img.shape[1]),np.uint8)
    bk_lr = np.zeros((img.shape[0]+(num_insert*2),num_insert),np.uint8)
    # decide gray value
    bk_tb[:,:] = grayvalue # white
    bk_lr[:,:] = grayvalue # white
    # insert pixel (top, bottom)
    array = np.insert(img, 0, bk_tb, axis=0)
    array = np.insert(array, array.shape[0], bk_tb, axis=0)

    # insert pixel (left, right)
    array = np.insert(array, [0], bk_lr, axis=1)
    array = np.insert(array, [array.shape[1]], bk_lr, axis=1)
    
    return np.array(array)

    # # the number of dimension applying FFT
    # N = 3 #(N * N)
    # # output(test)
    # img_combined_out = './sample/data1combined_ex.png'
    # cv2.imwrite(img_combined_out, np.array(array))
def FFT(img, num):
    amp = []
    phase = []
    num_insert = int((num-1)/2)
    h = img.shape[0] - (num_insert*2)
    w = img.shape[1] - (num_insert*2)
    for  y in range(num_insert,img.shape[0]-num_insert):
        for x in range(num_insert,img.shape[1]-num_insert):
            img1 = img[(y-num_insert):(y+num_insert+1),(x-num_insert):(x+num_insert+1)]
            fimg = np.fft.fft2(img1)
#             fshift = np.fft.fftshift(fimg)
            a = np.abs(fimg)
            p = np.angle(fimg)
#             a = np.abs(fshift)
#             p = np.angle(fshift)
            amp.append(a[num_insert, num_insert]) # centre coordinates (e.g. 5X5([0,0]~[4,4]) ->[2,2])
            phase.append(p[num_insert, num_insert]) # centre coordinates (e.g. 5X5([0,0]~[4,4]) ->[2,2])
    amp = np.reshape(amp,(h,w))
    phase = np.reshape(phase,(h,w))
    return amp, phase

### detect edge ###
# position 0: white 1: part of shading and edge 2: part of shading and not edge 
#                   3: part of symbol and edge  4: part of symbol and not edge 
def detect_edge(array,shading,txt,txt_ad):
    edge =[]
    num = 1
    for y in range(num,array.shape[0]-num):
        for x in range(num,array.shape[1]-num):
            if array[y,x] == 255:
                edge.append(0)
            elif (array[y,x] == 0) and (array[y,x] == shading[y-num,x-num]) and (array[y,x] != txt[y-num,x-num]):
                total_count = 0
                for i,j in itertools.product(range(-1,2),range(-1,2)):
                    if array[y+i,x+j] == 255:
                        total_count += 1
                if total_count >= 1:
                    edge.append(1)
                else:
                    edge.append(2)
            elif (array[y,x] == 0) and (array[y,x] != shading[y-num,x-num]) and (array[y,x] == txt[y-num,x-num]):
                total_count1 = 0
                for k,l in itertools.product(range(-1,2),range(-1,2)):
                    if txt_ad[y+k,x+l] == 255:
                        total_count1 += 1
                if total_count1 >= 1:
                    edge.append(3)
                else:
                    edge.append(4)
            else:
                total_count2 = 0
                for m,n in itertools.product(range(-1,2),range(-1,2)):
                    if txt_ad[y+m,x+n] == 255:
#                     if array[y+m,x+n] == 255:
                        total_count2 += 1
                if total_count2 >= 1:
                    edge.append(3)
                else:
                    edge.append(4)
    return edge

In [66]:
tic()
import os
# assign the area applying FFT
num = 61 # num * num
ck_gv_grid = 3 # check gray value 3X3 grid
os.makedirs('./output'+str(num)+'_csv',exist_ok=True) # create directory
number_of_data = 200
for i in range(number_of_data):
    # load image
    img_shade = cv2.imread('../test/output/data'+str(i+1)+'shading.png')
    img_text = cv2.imread('../test/output/data'+str(i+1)+'text.png')
    img_combined = cv2.imread('../test/output/data'+str(i+1)+'combined.png')
#     img_shade = cv2.imread('./sample/data1shading.png')
#     img_text = cv2.imread('./sample/data1text.png')
#     img_combined = cv2.imread('./sample/data1combined.png')

    # get information per pixel 
    gray_shade = cv2.cvtColor(img_shade,cv2.COLOR_RGB2GRAY)
    gray_text = cv2.cvtColor(img_text,cv2.COLOR_RGB2GRAY)
    gray_combined = cv2.cvtColor(img_combined,cv2.COLOR_RGB2GRAY)


    # add frame for FFT
    gray_combined_ex = addframe(num,255,gray_combined)
    amp,phase = FFT(gray_combined_ex, num)
    
    # add frame for distance to the edge ## update add predictors
    gray_combined_ad = addframe(ck_gv_grid,255,gray_combined)
    gray_text_ad = addframe(ck_gv_grid,255,gray_text)
    edge = detect_edge(gray_combined_ad, gray_shade, gray_text, gray_text_ad)
    edge = np.reshape(edge, (gray_combined.shape[0],gray_combined.shape[1]))

    # get image size
    height, width = gray_combined.shape[:2]

    # initialise
    data =[]
### add FFT value as predictors ###
#     for y in range(height):
#         for x in range(width):
#             if gray_combined[y,x] == 255:
#                 data.append([y,x,gray_combined[y,x],amp[(y*width)+x],phase[(y*width)+x],0])
#             elif (gray_combined[y,x] == 0) & (gray_combined[y,x] == gray_shade[y,x]) & (gray_combined[y,x] != gray_text[y,x]):
#                 data.append([y,x,gray_combined[y,x],amp[(y*width)+x],phase[(y*width)+x],1])
#             elif (gray_combined[y,x] == 0) & (gray_combined[y,x] != gray_shade[y,x]) & (gray_combined[y,x] == gray_text[y,x]):
#                 data.append([y,x,gray_combined[y,x],amp[(y*width)+x],phase[(y*width)+x],2])
#             else:
#                 data.append([y,x,gray_combined[y,x],amp[(y*width)+x],phase[(y*width)+x],2])
####################################

### PGM update add FFT value, distance to the edge as predictors ###
# label 0: white 1: part of shading 2: part of symbol
    for y in range(height):
        for x in range(width):
            if gray_combined[y,x] == 255:
                data.append([y,x,gray_combined[y,x],amp[y,x],phase[y,x],edge[y,x],0])
            elif (gray_combined[y,x] == 0) and (gray_combined[y,x] == gray_shade[y,x]) and (gray_combined[y,x] != gray_text[y,x]):
                data.append([y,x,gray_combined[y,x],amp[y,x],phase[y,x],edge[y,x],1])
            elif (gray_combined[y,x] == 0) and (gray_combined[y,x] != gray_shade[y,x]) and (gray_combined[y,x] == gray_text[y,x]):
                data.append([y,x,gray_combined[y,x],amp[y,x],phase[y,x],edge[y,x],2])
            else:
                data.append([y,x,gray_combined[y,x],amp[y,x],phase[y,x],edge[y,x],2])
####################################################################

    temp = np.array(data)
    df = pd.DataFrame(temp, columns =['y', 'x','gray_value','amp','phase','position', 'label'])
#     df.to_excel('./output_xlsx/data'+str(i+1)+'.xlsx', index=None)
    df.to_csv('./output'+str(num)+'_csv/data'+str(i+1)+'.csv', index=False,)
    print('Create data ' +str(i+1))
toc()



Create data 1
Create data 2
Create data 3
Create data 4
Create data 5
Create data 6
Create data 7
Create data 8
Create data 9
Create data 10
Create data 11
Create data 12
Create data 13
Create data 14
Create data 15
Create data 16
Create data 17
Create data 18
Create data 19
Create data 20
Create data 21
Create data 22
Create data 23
Create data 24
Create data 25
Create data 26
Create data 27
Create data 28
Create data 29
Create data 30
Create data 31
Create data 32
Create data 33
Create data 34
Create data 35
Create data 36
Create data 37
Create data 38
Create data 39
Create data 40
Create data 41
Create data 42
Create data 43
Create data 44
Create data 45
Create data 46
Create data 47
Create data 48
Create data 49
Create data 50
Create data 51
Create data 52
Create data 53
Create data 54
Create data 55
Create data 56
Create data 57
Create data 58
Create data 59
Create data 60
Create data 61
Create data 62
Create data 63
Create data 64
Create data 65
Create data 66
Create data 67
Crea

KeyboardInterrupt: 

In [3]:
# for counting time
import time

def tic():
    #require  to import time
    global start_time_tictoc
    start_time_tictoc = time.time()


def toc(tag="elapsed time"):
    if "start_time_tictoc" in globals():
        print("{}: {:.9f} [sec]".format(tag, time.time() - start_time_tictoc))
    else:
        print("tic has not been called")


In [65]:
#### FFT unit test ####
import itertools
num = 5
img = cv2.imread('../test/output/data1combined.png',0)
img_ex = addframe(num,255,img)
amp = []
phase = []
num_insert = 2 #int((num-1)/2)
for  y in range(num_insert,img_ex.shape[0]-num_insert):
    for x in range(num_insert,img_ex.shape[1]-num_insert):
        img1 = img_ex[(y-num_insert):(y+num_insert+1),(x-num_insert):(x+num_insert+1)]
        fimg = np.fft.fft2(img1)
        #fshift = np.fft.fftshift(fimg)
        a = np.abs(fimg)
        p = np.angle(fimg)
#         print(y,x)
#         print(fimg)
#         print(a)
#         print(img1.shape)
        for i,j in itertools.product(range(5),range(5)):
            amp.append(a[i,j])
            phase.append(p[i,j])
#             amp.append(a[num_insert, num_insert])
#             phase.append(p[num_insert, num_insert])
# amp = np.reshape(amp,(img.shape[0],img.shape[1]))
# amp.shape

3255000

In [45]:
#### detect distance unittest ####
# num = 3
# img1 = cv2.imread('../test/output/data1combined.png',0)
# img2 = cv2.imread('../test/output/data1shading.png',0)
# img3 = cv2.imread('../test/output/data1text.png',0)
# img_ex = addframe(num,255,img1)
# img_ex_txt = addframe(num,255,img3)
# edge = detect_edge(img_ex,img2,img3,img_ex_txt)
# # amp = np.reshape(amp,(img.shape[0],img.shape[1]))
# edge = np.reshape(edge,(img1.shape[0],img.shape[1]))
# np.savetxt('./test_edge_detect.csv',edge, fmt ='%d', delimiter=',')